In [1]:
import sys
import os
from os.path import join
from pathlib import Path
sys.path.append(join(Path(os.getcwd()).parent, 'lib'))

In [2]:
! python3 -m pip install unidecode

You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
%matplotlib inline

import json
import os
import pandas as pd
import re
import numpy as np
import pickle
import collections
from importlib import reload
from  mainlib import fasorial, lista, sdict, mm, mm_sep
from unidecode import unidecode
from of_proc import *
from os.path import join
from os import listdir as ld
from unidecode import unidecode
from matplotlib import pyplot as plt

Using TensorFlow backend.


In [4]:
def read_info(path, tem_perfil_funcionamento):
    global PATH, OUTPUT_PATH, PROCESSED_PATH, PERFIS_PATH, HORARIOS_PATH, PATH_MM, PATH_FAS, \
    DEG, MOR, PER, FAS, OUTPUT_HEADER,FAS_CONCAT, MM_CONCAT, PICKLES_CONCAT, OUT_INDEX, COLS_LISTA, \
    OUT, DATA_DIR, LISTA_CAD_PATH, PROCESSADOS_PICKLE, ERROS_PICKLE, LOGS_DIR, ADJACENCIAS_PATH
    with open(path) as f:
        info = json.load(f)
        PATH_FAS = info['paths']['fasoriais2']['input']
        PERFIS_PATH = info['paths']['perfis']['perfis']
        HORARIOS_PATH = info['paths']['perfis']['horarios']
        PATH_MM = info['paths']['mm']['input']
        DEG = info['paths']['nn_models']['degrau']
        MOR = info['paths']['nn_models']['morro']
        PER = info['paths']['nn_models']['perfil']
        FAS = info['paths']['nn_models']['fasorial']
        COLS_LISTA = info['lista2']['columns']  
        OUTPUT_HEADER = info['output']['header']
        OUT_INDEX = info['output']['header']
        OUT = info['paths']['output']
        DATA_DIR = info['paths']['base_dir']
        LISTA_CAD_PATH = info['paths']['lista-set-1']
        PROCESSADOS_PICKLE = info['paths']['processados']
        ERROS_PICKLE = info['paths']['erros']
        LOGS_DIR = info['paths']['logs']
        if tem_perfil_funcionamento:
            ADJACENCIAS_PATH = info['paths']['adjacencias']


# Parâmetros

In [5]:
json_file = '../../info/info-LIGHT.json'
gd = False
tem_perfil_funcionamento = True

# Se for remover registros de tensões zeradas, configurar se preenche ou não os registros removidos
# Pra processamento EDP-ES = False. Para a Light = True
fill_rows = True

# Informações

In [6]:
def dk(d,k=0): return list(d.values())[k]

In [7]:
read_info(json_file, tem_perfil_funcionamento)

In [8]:
def get_horario_info2(json_file):
    global ATIV_PATH
    with open(json_file) as jinfo:
        info = json.load(jinfo)
        HORARIOS_PATH = info['paths']['perfis']['horarios']
        PERFIS_PATH = info['paths']['perfis']['perfis']
        ATIV_PATH = info['paths']['perfis']['mapa']


    perfis = pd.read_csv(PERFIS_PATH, sep = ';')
    ativs = pd.read_csv(ATIV_PATH, sep = ';')

    #perfis.columns = ['id', 'instalacao', 'cpf_cnpj', 'razao social', 'endereco_cadastro', 'ramo_de_atividade','None1', 'None2']
    
    dates = re.compile("[0-9]{2}:[0-9]{2}")
    dateparser = lambda x: pd.datetime.strptime(x, '%H:%M') if type(x) == str  and dates.match(x) else pd.NaT
    cols = ['name','inicio_dia_util','fim_dia_util', 'inicio_sabado', 'fim_sabado', 'inicio_domingo', 'fim_domingo', 'inicio_feriado', 'fim_feriado']
    horarios = pd.read_csv(HORARIOS_PATH, sep = ';',skipinitialspace=True, skiprows = 2, na_values = np.NaN, names = cols,
                          date_parser = dateparser, parse_dates = [1,2,3,4,5,6,7,8])
    
    
    horarios.set_index('name', inplace = True)
    horarios.dropna(inplace=True)

    perfis.set_index('instalacao', inplace=True)
    perfis = (perfis.groupby(level=0).first())

    
    return horarios, perfis, ativs

def get_files_4(nomes, dim,  CAMINHO):
    not_found = []
    dif = {}
    for i in nomes:
        if i in dim:
            tmp_list = []
            search = ab.search_list_2(str(i),os.listdir(CAMINHO))
            if search:
                dif[i] = search
            else:
                not_found.append(i)                
    return dif, not_found

#Criar dicionario de categorias
def insta2horario(HORARIOS_PATH):
    horarios = pd.read_csv(HORARIOS_PATH, sep=',')
    horarios.drop(columns=horarios.columns[0],inplace=True)
    horarios.columns = ['nome', 'cat', 'dias_uteis_i',
           'dias_uteis_f', 'sabado_i', 'sabado_f', 'domingo_i',
           'domingo_f', 'feriado_i', 'feriado_f']
    horarios['nome'] = horarios['nome'].apply(unidecode)
    dfcat = horarios.iloc[:,:2]
    dfcat.set_index('nome',inplace=True)
    scat = dfcat['cat']
    dcat = scat.to_dict()
    horarios.set_index('nome', inplace=True)
    func = {k: [pd.Timestamp(v) for v in horarios.loc[k,'dias_uteis_i':'feriado_f']] for k in horarios.index}
    return func

def per2cluster(func, ATIV):
    dfativ = pd.read_csv(ATIV, sep=';', names = ['ramo', 'perfil'])
    dfativ=dfativ.applymap(unidecode)
    dfativ.set_index('ramo', inplace=True)
    a2p = dfativ.loc[:,'perfil'].T.to_dict()
    a2p.pop('Ramo de Atividade')
    p2a = dict((v,k) for k,v in a2p.items())
    perfunc=PERFIS['ramo_de_atividade'].map(a2p).map(func)
    return perfunc

In [9]:
global df_mm_new
mmm = mm(json_file)
df_mm_new = pd.read_csv(PATH_MM,date_parser = mmm.dateparser, index_col=[0], sep=';')
dummy_mm = ""

## l_______________________________________________________________________

### Carregar modelos neurais

# Coleções principais

In [10]:
global LISTA_CAD
global PERFIL, DEGRAU, MORRO, NN_FAS
global files_fas, files_mm
global HORARIOS, PERFIS, ATIVS, DPERFIS, DPERFIS_NORMAL, DPERFIS_TROCA
global dim, dmi, df_fas, dfiles_mm, dfiles_fas_troca, dfiles_mm_troca, dfiles_fas_normal, dfiles_mm_normal
global o_i_normal, o_i_troca, o_instas, list_i_normal, list_i_troca, list_intas, gd

In [11]:
PERFIL, DEGRAU, MORRO, NN_FAS = load_models(PER, DEG, MOR, FAS)

Instructions for updating:
Colocations handled automatically by placer.


In [12]:
LISTA_CAD_PATH

'/home/ubuntu/data/data/maio_2019/setembro/lista-cad-pnt-20190820.csv'

## l_______________________________________________________________________

In [13]:
try:
    LISTAF = pd.read_csv(LISTA_CAD_PATH, names = COLS_LISTA, header=0, sep=';',engine='python')
except:
    LISTAF = pd.read_csv(LISTA_CAD_PATH, names = COLS_LISTA, header=0, engine='python')

In [14]:
LISTAF['instalacao'] = pd.DataFrame(LISTAF['instalacao'].apply(lambda x: '{0:0>10}'.format(x)))
try:
    LISTAF['medidor'] = pd.DataFrame(LISTAF['medidor'].astype(int).apply(lambda x: '{0:0>8}'.format(x)))
except:    
    LISTAF['medidor'] = pd.DataFrame(LISTAF['medidor'].apply(lambda x: '{0:0>8}'.format(x)))


In [15]:
# LIGHT
faltantes = [
'BT2A411444175'    
]

In [16]:
LISTAF = LISTAF[LISTAF['instalacao'].isin(faltantes)]

In [17]:
LISTA_CAD = get_list(json_file)
LISTA_CAD.df =  LISTAF
if tem_perfil_funcionamento:
    HORARIOS, PERFIS, ATIVS = get_horario_info2(json_file)
dim, dmi = dicts(LISTA_CAD)

#### Estruturas auxiliares

In [18]:
list_columns_remove = ['cliente', 'medidor', 'data_inicio', 'data_fim', 'lote', 'ssn', 'troca']

In [19]:
cols = list(LISTA_CAD.df.columns[~LISTA_CAD.df.columns.isin(list_columns_remove)==True])

In [20]:
del(cols[1])

In [21]:
LISTA_CAD.df.shape

(1, 12)

In [22]:
cols_without_instal = cols.copy()
del(cols_without_instal[0])

In [23]:
troca = False
o_i_normal = collections.OrderedDict(sorted(LISTA_CAD.mes_days(troca, cols).items()))
troca = True
o_i_troca = collections.OrderedDict(sorted(LISTA_CAD.mes_days(troca, cols).items()))
o_instas =  collections.OrderedDict(sorted({**o_i_normal, **o_i_troca}.items()))

#### Arquivos

In [24]:
import A_base
reload(A_base)

<module 'A_base' from '/home/ubuntu/data/code/ref_fdetector/src/lib/A_base.py'>

In [25]:
import A_base as ab

In [26]:
dfiles_fas, nf_fas= get_files_4(list(o_instas.keys()), dim,  PATH_FAS)
ddirs_fas = {k:join(PATH_FAS,v) for k,v in dfiles_fas.items()}

#### Lista , perfis e horários

In [27]:
# Dicionário de instalacao para horario de funcionamento
if tem_perfil_funcionamento:
    func =  insta2horario(HORARIOS_PATH)
    # Dicionário de perfis para clusters
    perfunc = per2cluster(func, ATIV_PATH)
    percats = PERFIS['cat'].fillna(value=0).apply(int).to_dict()

 l___________________________________________________________________________________________________________________________________________

#### Pickles

# Importante

In [28]:
log_count=0

In [29]:
sem_hora = []
err_msg = {}
missing = []

In [30]:
with open(PROCESSADOS_PICKLE,'rb') as p:
    processados=pickle.load(p)
with open(ERROS_PICKLE,'rb') as p:
    erros = pickle.load(p)

In [31]:
os.path.join(LOGS_DIR,str(log_count))

'../../logs/0'

In [32]:
log_file = open(os.path.join(LOGS_DIR,str(log_count)), 'at+')

In [33]:
def log(string):
    print(string)
    log_file.write(string)

In [34]:
def dici2list(d_insta_to_days):
    ddays={}
    ddays[4],ddays[5],ddays[6],ddays[7] = d_insta_to_days[12], d_insta_to_days[1], d_insta_to_days[2], d_insta_to_days[3], d_insta_to_days[4]
    return ddays
                

In [35]:
import traceback

In [36]:
pd.options.mode.chained_assignment = None  # default='warn' . option None

In [37]:
padrao = [pd.Timestamp('2019-09-20 00:00:00'),
 pd.Timestamp('2019-09-20 23:59:00'),
 pd.Timestamp('2019-09-20 00:00:00'),
 pd.Timestamp('2019-09-20 23:59:00'),
 pd.Timestamp('2019-09-20 00:00:00'),
 pd.Timestamp('2019-09-20 23:59:00'),
 pd.Timestamp('2019-09-20 00:00:00'),
 pd.Timestamp('2019-09-20 23:59:00')]

In [38]:
def loopy2(d_insta_to_days, d_insta_to_files_fas, d_insta_to_files_mm,PATH_FAS, PATH_MM, output_dir,sepfas, sepmm, gd, tem_perfil_funcionamento, fill_rows):
    df_out = pd.DataFrame(index=OUT_INDEX)
    global difas, dimm
    global dfas, dmm,  dimm
    global threshold_tensoes_zeradas, df_tensoes_zeradas
    difas, dimm = sdict(), sdict()
    c = 0
    c1 = 0
    df_tensoes_zeradas = [pd.DataFrame(columns=['Mes', 'Instalacao'])]
    threshold_tensoes_zeradas = 0.5
    if tem_perfil_funcionamento:
        with open(ADJACENCIAS_PATH,'rb') as p:
            adjacencias_df = pickle.load(p)
    else:
        adjacencias_df = []
    for instalacao in d_insta_to_days:
        c1 = c1 + 1
        inst_year = o_instas[instalacao][cols_without_instal[0]].year
        if (instalacao  not in processados):
            log(str(c)+'--'+'######## ' + instalacao + ' #########')
            medidor = dim[instalacao][0] if type(dim[instalacao]) is list else dim[instalacao]
            try:      
                print('------ 0')
                if tem_perfil_funcionamento:
                    ramo = percats[instalacao] if instalacao in percats else 0
                else:
                    ramo = 0

                if ramo == 0:
                    profile_err = True
                else:
                    profile_err = False

                print('------ 1')
                if tem_perfil_funcionamento:
                    hours = perfunc[instalacao] if instalacao in perfunc else padrao
                else:    
                    hours = padrao
                print('------ 2')
                ddays = d_insta_to_days[instalacao] if instalacao in d_insta_to_days else default_ddays_2()
                print('------ 3')
                dfas = fasorial(json_file, name = instalacao, ddays=ddays)     
                print('------ 4')
                dfas.outputs['perfil'] = ramo
                print('------ 5')
                dfas.read_fasorial(d_insta_to_files_fas[instalacao], dtype=None, index_col=[5], sep = sepfas)
                print('------ 6')
                dmm = mm(json_file, name = instalacao, ddays = ddays, ramo=ramo)
                print('------ 7')
                if dim[instalacao] in df_mm_new.columns:
                    dmm.df = pd.DataFrame(df_mm_new[dim[instalacao]])
                else:
                    dmm.df = pd.DataFrame(df_mm_new[str(int(dim[instalacao]))])                 

                print('------ 8')
                proc_fas(dfas,ddays, NN_FAS, json_file, hours = dfas.hours, gd=gd, fill_rows=fill_rows, threshold_tensoes_zeradas = threshold_tensoes_zeradas, df_tensoes_zeradas = df_tensoes_zeradas)
                processa_mm(dmm, ddays, PERFIL, DEGRAU, MORRO, json_file, adjacencias_df, tem_perfil_funcionamento, profile_err)
                print(d_insta_to_files_fas[instalacao])
                print('################################################')
                #Cria um dicionário de outputs por mês para o fasorial
                for y in dfas.months:
                    dfas.outputs[y] = {}
                    for month in dfas.months[y]:
                        if len(df_tensoes_zeradas[0][(df_tensoes_zeradas[0]['Mes']==month) & (df_tensoes_zeradas[0]['Instalacao']==dfas.name)]) == 0:
                            dfas.outputs[y][month] = dfas.months[y][month].outputs 
                df_out = cat_out2(df_out, dmm.outputs, dfas.outputs, ddays,OUT_INDEX, cols_without_instal, inst_year)
                processados.append(instalacao)
                difas[instalacao] = dfas
                dimm[instalacao] = dmm                
                c = c+1                
            except Exception as e:
                erros.append(instalacao)
                err_msg[instalacao] = traceback.format_exc()
                print('++++++Erro 2++++++++++ \n'+ str(traceback.format_exc())+'\n++++++++++++++++++++')
    df_out = df_out.T
    df_out.to_csv(output_dir + 'new.csv')
    for index, row in df_tensoes_zeradas[0].iterrows():
        index_to_drop = df_out[(df_out['Nome'] == row['Instalacao']) & (df_out['Mês'] == row['Mes'])].index
        df_out.drop(index_to_drop , inplace=True)
    df_tensoes_zeradas[0].to_csv(output_dir + 'tensoes_zeradas.csv') 
    df_out.to_csv(output_dir + 'newAfter.csv')

## l_______________________________________________________________________

In [39]:
from mainlib import mm

In [40]:
processados, erros = [], []
sepmm=','
sepfas=';'
m=loopy2(o_instas, ddirs_fas, dummy_mm, PATH_FAS, PATH_MM, OUT, sepfas, sepmm, gd, tem_perfil_funcionamento, fill_rows)

0--######## BT2A411444175 #########
------ 0
------ 1
------ 2
------ 3
------ 4
------ 5
------ 6
------ 7
------ 8
{5: Timestamp('2019-05-13 00:00:00'), 6: Timestamp('2019-06-13 00:00:00'), 7: Timestamp('2019-07-11 00:00:00')}
Registros com Tensão Zerada =  0
Total de Registros =  8631
Proporcao de Zerados =  0.0 %
Possui Tensao Zerada =  False
Possui Tensao Zerada =  False


/home/ubuntu/data/code/ref_fdetector/src/lib/mainlib.py:766: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[(f1 & f2) | f3]


Registros com Tensão Zerada =  0
Total de Registros =  8631
Proporcao de Zerados =  0.0 %
Possui Tensao Zerada =  False
Possui Tensao Zerada =  False


/home/ubuntu/data/code/ref_fdetector/src/lib/mainlib.py:766: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[(f1 & f2) | f3]


Registros com Tensão Zerada =  0
Total de Registros =  8634
Proporcao de Zerados =  0.0 %
Possui Tensao Zerada =  False
Possui Tensao Zerada =  False
{5: Timestamp('2019-05-13 00:00:00'), 6: Timestamp('2019-06-13 00:00:00'), 7: Timestamp('2019-07-11 00:00:00')}
{2019: {6: <mainlib.mm object at 0x7f58295183c8>, 7: <mainlib.mm object at 0x7f5800c2e2e8>}}
Ano/Mês =  2019 / 5
Perfil Cadastrado   =  1
Perfil Classificado =  3
Severidade =  1
Ano/Mês =  2019 / 6
Perfil Cadastrado   =  1
Perfil Classificado =  3
Severidade =  1
Ano/Mês =  2019 / 7
Perfil Cadastrado   =  1
Perfil Classificado =  3
Severidade =  1
/home/ubuntu/data/data/maio_2019/setembro/fas-sep/BT2A411444175.csv
################################################


/home/ubuntu/data/code/ref_fdetector/src/lib/mainlib.py:766: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[(f1 & f2) | f3]


In [44]:
dfas.months[2019][7].df[dfas.months[2019][7].df['angcheck'] == True][['SEQ','AVA','AVB','AVC','aab','aac','abc', 'angcheck']]

,SEQ,AVA,AVB,AVC,aab,aac,abc,angcheck
data,,,,,,,,
2019-06-25 21:30:00,127.0,0.000,60.000,120.000,60.0,120.0,60.0,True
2019-07-01 05:15:00,NaN,0.001,0.001,0.001,0.0,0.0,0.0,True
2019-07-01 05:30:00,NaN,0.001,0.001,0.001,0.0,0.0,0.0,True
2019-07-01 05:45:00,NaN,0.001,0.001,0.001,0.0,0.0,0.0,True


##### Substituir coluna "Perfil cadastrado" por texto

In [ ]:
#l=LISTA_CAD.df.drop(index=0)

In [ ]:
if tem_perfil_funcionamento:
    ra=PERFIS['ramo_de_atividade']
    dra=ra.to_dict()
    o = pd.read_csv(OUT + 'new.csv')
    o['Perfil cadastrado']=o['Nome'].map(dra)
    o.to_csv(OUT + 'new2.csv')

---

# Contar vazios mês (cvm)

### Parâmetros

In [ ]:
arquivo_faltantes_path = '../../outputs/'
fas_nome_arquivo_faltantes = 'fas_vazios_EDP-ES.csv'
mm_nome_arquivo_faltantes = 'mm_vazios_EDP-ES.csv'
# EDP-ES usa ';', LIGHT = ','
sep = ';'

______________________________________________________________________________________________________________________________________________

In [ ]:
! python3 -m pip install unidecode

In [ ]:
import sys
import os
from os.path import join
from os import listdir as ld
sys.path.append(join('/',*os.getcwd().split('/')[:-1], 'lib'))

In [ ]:
%matplotlib inline
import pandas as pd
import of_proc
import mainlib
from os import listdir as ld
from mainlib import dframe,fasorial
from matplotlib import pyplot as plt
from of_proc import pdump, pload, dk, daterange

______________________________________________________________________________________________________________________________________________

In [ ]:
data = dframe(json_file).data
dp = dframe(json_file).dateparser

In [ ]:
lcols = data['lista2']['columns']

In [ ]:
all_months = lcols[7:11]

In [ ]:
last_three_months = lcols[8:11]

In [ ]:
date_fields = all_months

In [ ]:
dateparser = lambda x: pd.datetime.strptime(x, '%d/%m/%Y')

In [ ]:
try:
    LISTAF = pd.read_csv(LISTA_CAD_PATH, names = COLS_LISTA, header=0, parse_dates=date_fields, date_parser=dateparser, sep=';',engine='python')
except:
    LISTAF = pd.read_csv(LISTA_CAD_PATH, names = COLS_LISTA, header=0, parse_dates=date_fields, date_parser=dateparser, engine='python')

In [ ]:
lista = LISTAF

In [ ]:
lista.set_index('instalacao', inplace=True)

In [ ]:
lt = lista[lista['troca']=='2-Com troca de medidor']

In [ ]:
dim = lista['medidor'].groupby(level=0).apply(list)

----

________________________________________________________________________________________________________________________________________________

#### Leituras faltando - Fasorial

_________________________________________________________________________________________________________________________________________________

In [ ]:
def get_month(df, date, tolerance=10):
    begin_date = date - pd.Timedelta(days=30)
    success = False
    count = 0
    while not success and count <= tolerance:
        try:
            dfo = df[begin_date:date]
            success=True
            return dfo
        except:
            begin_date = begin_date + pd.Timedelta(days=1)
            count = count+1
    else:
        print('Mes nao preenchido')
        return None

In [ ]:
lista.info()

In [ ]:
faso = fasorial(json_file)
erros = []
emp = {}
emp_porc = {}
faso=fasorial(json_file)
print(len(ddirs_fas))
months = {}
total_rows = 2880
for n,(k,v) in enumerate(ddirs_fas.items()):
    print(n, end=' ')
    if True:
        emp[k]={}
        if k.isdigit():
            l = lista[all_months].loc[int(k)]
        else:
            l = lista[all_months].loc[k]
        if type(l) == pd.Series:
            ddays = dict(l)        
        elif type(l) == pd.DataFrame:
            ddays = dict(l.groupby(level=0).first().iloc[0])
        else:
            print('Erro desconhecido: ', k, type(ddays), ddays)
        dff = pd.read_csv(v,sep=';',date_parser = dp,parse_dates=[5],names=data['fasorial']['columns'],index_col=5)
        dff.sort_index(inplace=True)
        for k1,v1 in ddays.items():
            months[k1] = get_month(dff, v1)
        for k1,v1 in months.items():
            months_rows = len(v1)
            missing_rows = total_rows - months_rows
            if missing_rows > 0:
                emp[k][k1] = missing_rows
            else:
                emp[k][k1] = 0
            #emp[k][k1]  = v1[0].resample('15min').first().isna().all(axis=1).sum()
    else:
        pass
        #print(e)
        #erros.append(k)

_______________________________________________________________________

In [ ]:
dmi={}
for k,v in dim.items():
    for i in v:
        dmi[int(i)] = k

In [ ]:
dempt=pd.DataFrame(index=emp.keys(), data=emp.values(),columns = last_three_months)

In [ ]:
dempt.to_csv(arquivo_faltantes_path + fas_nome_arquivo_faltantes)

_______________________________________________________________________

#### Testes Resultado Fasorial

_______________________________________________________________________

### Leituras faltando -  Memória de massa

_______________________________________________________________________

In [ ]:
MM = PATH_MM

In [ ]:
MM

In [ ]:
m = pd.read_csv(MM, sep=';',decimal=',', low_memory=False, index_col = [0], na_values=' ', date_parser = dp, parse_dates=[0])

In [ ]:
m.sort_values(by=['DATA/Hora'], inplace=True)

In [ ]:
#m = m.applymap(float)

In [ ]:
mt = m.T

In [ ]:
mt.index = pd.Series(mt.index).apply(float).apply(int).values

In [ ]:
mt.insert(loc=0, value = mt.index.map(dmi).values, column = 'instalacao')

In [ ]:
try:
    mt.index = mt['instalacao'].apply(int)
except:
    mt.index = mt['instalacao']

In [ ]:
mt.drop(columns='instalacao', inplace=True)

In [ ]:
mt2 = mt.groupby(level=0).max()

In [ ]:
m2 = mt2.T

In [ ]:
m2.index=pd.to_datetime(m2.index)

_______________________________________________________________________

In [ ]:
def get_month_2(df, date, tolerance=10):
    begin_date = date - pd.Timedelta(days=30)
    success = False
    count = 0
    while not success and count <= tolerance:
        try:
            dfo = df[begin_date:date]
            success=True
            return dfo
        except:
            begin_date = begin_date + pd.Timedelta(days=1)
            count = count+1
    else:
        print('Mes nao preenchido')
        return None

_______________________________________________________________________

In [ ]:
ddays = lista.groupby(level=0)[all_months].first().T.to_dict()

In [ ]:
dd = {k:list(v.values()) for k,v in ddays.items()}

In [ ]:
ddays = dd

In [ ]:
mem_m = {}
for c in m2.columns:
    mem_m[c]={}
    for d in ddays[c]:
        mem_m[c][d.month] = get_month_2(m2[c], d)


In [ ]:
em = {}
for k,v in mem_m.items():
    em[k] = {}
    for k1,v1 in v.items():
        if v1 is not None:
            #em[k][k1] = v1.resample('H').max().notnull().sum()
            em[k][k1] = v1.resample('H').min().isna().sum()

In [ ]:
df = pd.DataFrame()
for k,v in em.items():
    df = pd.concat( (df, pd.DataFrame(index=v.keys(), data=v.values(), columns=[k]).T))

In [ ]:
df = df[all_months]

In [ ]:
df.index.name = 'mês'

In [ ]:
df2=df.applymap(lambda x: max(0, x))

In [ ]:
df2t = df2.T.applymap(int)

In [ ]:
df2t

In [ ]:
df2t.to_csv(arquivo_faltantes_path + mm_nome_arquivo_faltantes)

_______________________________________________________________________

#### Testes Resultado Memória de Massa